In [11]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
import sqlite3 as sq

# Archive #4 (?)

#### Procedure for reading the data

We used SQLite to process the file.

In [10]:
Member = pd.merge(Member, Genes, how="left", left_on="gene_id", right_on="id")

Member = pd.merge(Member, )

In [9]:
conn = sq.connect('yeast.sqlite')

In [8]:
tables_import = ['Member', 'Genes', 'locations', 'Molecular', 'Bioprocess']
for table in tables_import:
    exec(f'{table} = pd.read_sql_query(f"SELECT * from {table}", sq.connect("yeast.sqlite"))')